In [ ]:
from typing import Tuple
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tqdm.auto import tqdm

import torch
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

#read data

Traindf = pd.read_csv("/content/drive/MyDrive/fashion-mnist_train.csv")
Testdf = pd.read_csv("/content/drive/MyDrive/fashion-mnist_test.csv")


In [ ]:
Traindf


,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,0,0,105,92,101,107,100,132,0,0,2,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,150,...,211,220,214,74,0,255,222,128,0,0,0,0,0,0,0,0,0,44,12,0,0,40,134,162,191,214,163,146,165,79,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,0,114,183,112,55,23,72,102,165,160,28,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,24,188,163,93,...,171,249,207,197,202,45,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,22,21,25,69,52,45,74,39,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,0,0,0,46,0,21,68,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,187,189,...,230,237,229,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,68,116,112,136,147,144,121,102,63,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59996,1,0,0,0,0,0,0,0,0,0,0,83,155,136,116,148,110,118,67,32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,118,...,199,165,108,108,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,32,159,6,91,0,0,85,159,0,73,0,0,0,0,0,0,0,0,0
59997,8,0,0,0,0,0,0,0,0,0,0,1,0,0,87,114,77,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,...,228,229,231,231,218,230,255,0,0,0,0,0,0,0,0,0,0,0,116,140,147,166,176,174,173,173,174,173,177,164,160,162,163,135,94,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
Y_train=Traindf["label"]
Y_test =Testdf["label"]
X_train=Traindf.drop(["label"],axis = 1, inplace = False)
X_test=Testdf.drop(["label"],axis = 1, inplace = False)
Y_train=np.array(Y_train)
Y_test=np.array(Y_test)
x_train = np.array(X_train)
x_test =np.array(X_test)

In [ ]:
x_train=x_train.reshape(60000, 1,28,28)
x_test = x_test.reshape(10000, 1,28,28)

In [ ]:
x_train.shape


(60000, 1, 28, 28)

In [ ]:
#Pre_processing
x_train, x_test = x_train/255.0, x_test/255.0

In [ ]:
from torch.utils.data import DataLoader, Dataset, TensorDataset

In [ ]:
Y_train

array([2, 9, 6, ..., 8, 8, 7])

In [ ]:
y_train = Y_train
y_test = Y_test

In [ ]:
#Convert Data to PytorchData
#Way 1
train_ds = TensorDataset(torch.from_numpy(x_train), torch.from_numpy(y_train))
test_ds = TensorDataset(torch.from_numpy(x_test), torch.from_numpy(y_test))

In [ ]:
train_ds

In [ ]:
#Way 2 
class Data(Dataset):
    def __init__(self, x: np.array, y: np.array) -> None:
        self.x, self.y = x, y

    def __len__(self) -> int:
        return len(self.x)

    def __getitem__(self, i) -> Tuple[np.array, np.array]:
        return self.x[i], self.y[i]

In [ ]:
train_ds = Data(x_train.astype(np.float32), y_train.astype(np.int8))
test_ds = Data(x_test.astype(np.float32), y_test.astype(np.int8))

In [ ]:
train_ds

In [ ]:
type(train_ds[2][0]), train_ds[2][1].dtype

(numpy.ndarray, dtype('int8'))

In [ ]:
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True, drop_last=True, num_workers=4)
test_dl = DataLoader(test_ds, batch_size=64, shuffle=False, drop_last=False, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
train_dl

In [ ]:
#Check output of Dataloader
batch_x, batch_y = next(iter(train_dl)) 

batch_x.dtype, batch_x.shape, batch_y,batch_y.long()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


(torch.float32,
 torch.Size([64, 1, 28, 28]),
 tensor([3, 9, 8, 0, 6, 2, 9, 3, 0, 1, 0, 5, 3, 2, 3, 4, 1, 4, 3, 1, 8, 3, 9, 3,
         4, 7, 0, 6, 2, 3, 8, 4, 7, 8, 4, 2, 9, 4, 4, 9, 7, 4, 2, 3, 3, 8, 4, 7,
         5, 2, 8, 0, 0, 1, 7, 8, 1, 7, 7, 6, 5, 8, 1, 6], dtype=torch.int8),
 tensor([3, 9, 8, 0, 6, 2, 9, 3, 0, 1, 0, 5, 3, 2, 3, 4, 1, 4, 3, 1, 8, 3, 9, 3,
         4, 7, 0, 6, 2, 3, 8, 4, 7, 8, 4, 2, 9, 4, 4, 9, 7, 4, 2, 3, 3, 8, 4, 7,
         5, 2, 8, 0, 0, 1, 7, 8, 1, 7, 7, 6, 5, 8, 1, 6]))

In [ ]:
batch_x

tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..

In [ ]:
#Build Model 
import torch.nn as nn
import torch.nn.functional as F

CNN_model = nn.Sequential(
    nn.Conv2d(1, 10, kernel_size=3, stride=2),
    nn.ReLU(),
    nn.Conv2d(10, 20, kernel_size=3, stride=2),
    nn.ReLU(),
    nn.Conv2d(20, 40, kernel_size=3, stride=2),
    nn.ReLU(),
)
#3Conve2d 3 Relu

In [ ]:
CNN_model(batch_x).shape

torch.Size([64, 40, 2, 2])

In [ ]:
#Flatten data and Create Out put Dense
model = nn.Sequential(
    CNN_model,
    nn.Flatten(),
    nn.Linear(40*2*2, 10)
)

In [ ]:
#Loss
loss_fn = nn.CrossEntropyLoss()

In [ ]:
#Training 
EPOCHS = 3
LR = 0.01
for _ in range(EPOCHS):
    for batch_x, batch_y in tqdm(train_dl):
        loss = loss_fn(model(batch_x), batch_y.long())
        loss.backward()
        with torch.no_grad():
            for w in model.parameters():
                w -= LR * w.grad # w_new = w_old - LR * grad (Gradient Decent)
                w.grad = None
        print(f"\rLoss is {loss:.4f}", end="")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/937 [00:00<?, ?it/s]

Loss is 0.5380

  0%|          | 0/937 [00:00<?, ?it/s]

Loss is 0.5757

...

Loss is 0.5592

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f99356f0170>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f99356f0170>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

Loss is 0.5650

    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f99356f0170>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f99356f0170>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/

Loss is 0.4867

In [ ]:
#Check with TestData
y_preds, y_true = [], []
with torch.no_grad():
    for batch_x, batch_y in tqdm(test_dl):
        y_pred = model(batch_x).argmax(dim=-1)
        y_preds.append(y_pred)
        y_true.append(batch_y)

(torch.cat(y_preds) == torch.cat(y_true)).float().mean()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f99356f0170>
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f99356f0170>
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f99356f0170>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f99356f0170>
    self._shutdown_workers()
Traceback (most re

  0%|          | 0/157 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    if w.is_alive():
    self._shutdown_workers()
    self._shutdown_workers()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    assert self._parent_p

tensor(0.7742)